# Sentiment Analysis of Text Based Content

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

In [3]:
df = pd.read_csv('TwitterData.csv')
df.clean_text=df.clean_text.astype(str)

In [4]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [5]:
df.columns

Index(['clean_text', 'category'], dtype='object')

In [11]:
df = df[df['category'] != 0.0 ]
print(df.shape)
df.head(5)

(107767, 2)


,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
8,with upcoming election india saga going import...,1.0


In [12]:
df['category'] = df['category'].replace(to_replace={-1:'Negative',1:'Positive'}) 

In [13]:
df.dtypes

clean_text    object
category      object
dtype: object

In [14]:
df["category"].value_counts()

Positive    72250
Negative    35510
Name: category, dtype: int64

In [15]:
sentiment_label = df.category.factorize() #Converting categorical values to numeric values
sentiment_label

(array([0, 1, 1, ..., 0, 0, 1], dtype=int64),
 Index(['Negative', 'Positive'], dtype='object'))

In [16]:
#Tokenizing the words/sentences into small parts "Token"
tweet = df.clean_text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet) #creates an association between the words and the assigned numbers
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet) #replacing the words with their assigned number
padded_sequence = pad_sequences(encoded_docs, maxlen=200) #Use padding to pad the sentences to have equal length.

In [17]:
print(tokenizer.word_index)

{'modi': 1, 'the': 2, 'and': 3, 'for': 4, 'you': 5, 'are': 6, 'that': 7, 'not': 8, 'india': 9, 'this': 10, 'will': 11, 'with': 12, 'has': 13, 'have': 14, 'but': 15, 'all': 16, 'his': 17, 'was': 18, 'from': 19, 'they': 20, 'who': 21, 'people': 22, 'bjp': 23, 'your': 24, 'like': 25, 'what': 26, 'congress': 27, 'our': 28, 'now': 29, 'can': 30, 'only': 31, 'its': 32, 'why': 33, 'more': 34, 'govt': 35, 'narendra': 36, 'one': 37, 'about': 38, 'their': 39, 'him': 40, 'just': 41, 'vote': 42, 'country': 43, 'dont': 44, 'how': 45, 'when': 46, 'time': 47, 'years': 48, 'even': 49, 'any': 50, 'election': 51, 'rahul': 52, 'good': 53, 'indian': 54, 'nation': 55, 'which': 56, 'modis': 57, 'should': 58, 'then': 59, 'than': 60, 'did': 61, 'against': 62, 'there': 63, 'also': 64, 'after': 65, 'know': 66, 'out': 67, 'them': 68, 'get': 69, 'power': 70, 'money': 71, 'government': 72, 'because': 73, 'had': 74, 'these': 75, 'very': 76, 'other': 77, 'been': 78, 'want': 79, 'would': 80, 'again': 81, 'see': 82, '

In [18]:
print(tweet[0])
print(encoded_docs[0])

when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples
[46, 1, 341, 72, 1993, 885, 40, 2821, 2, 961, 205, 2, 208, 33, 164, 109, 48, 69, 1048, 208, 58, 3, 8, 540, 3, 58, 4281, 3, 2614]


In [19]:
print(padded_sequence[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   46    1  341   72 1993  885   40 2821    2  961  205
    2  208   33  164  109   48   69 1048  208   58    3    8  540    3
   58 

In [21]:
embedding_vector_length = 32
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])  
print(model.summary()) 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 32)           2969984   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 200, 32)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 50)                16600     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2,986,635
Trainable params: 2,986,635
Non-trainable params: 0
____________________________________________

In [22]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
2695/2695 [==============================] - 690s 255ms/step - loss: 0.3319 - accuracy: 0.8566 - val_loss: 0.2168 - val_accuracy: 0.9202
Epoch 2/5
2695/2695 [==============================] - 526s 195ms/step - loss: 0.2001 - accuracy: 0.9260 - val_loss: 0.1848 - val_accuracy: 0.9350
Epoch 3/5
2695/2695 [==============================] - 519s 192ms/step - loss: 0.1742 - accuracy: 0.9356 - val_loss: 0.1693 - val_accuracy: 0.9401
Epoch 4/5
2695/2695 [==============================] - 527s 195ms/step - loss: 0.1529 - accuracy: 0.9419 - val_loss: 0.1454 - val_accuracy: 0.9492
Epoch 5/5
2695/2695 [==============================] - 495s 184ms/step - loss: 0.1411 - accuracy: 0.9478 - val_loss: 0.1327 - val_accuracy: 0.9542


In [23]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print("Predicted label: ", sentiment_label[1][prediction])

In [30]:
test_sentence1 = "You are soo beautiful"
predict_sentiment(test_sentence1)

test_sentence2 = "You creep, just get lost from here!"
predict_sentiment(test_sentence2)

1/1 [==============================] - 0s 35ms/step
Predicted label:  Positive
1/1 [==============================] - 0s 33ms/step
Predicted label:  Negative
